# sat-stac tutorial: STAC Classes

This notebook contains examples of using satstac to work with STAC catalogs.

**Table of contents**

- <a href=#existing>Working with existing catalogs</a>
- <a href=#create>Creating new catalogs</a>
    - <a href="#addcat">Adding catalogs to catalogs</a>
    - <a href="#addcol">Adding collections to catalogs</a>
    - <a href="#additem">Adding items to collections</a>
- <a href=#views>Views (sub-catalogs)</a>


The examples here use the [test catalog in the sat-stac repo](https://github.com/developmentseed/sat-stac/tree/master/test/catalog). The directory structure of the test catalog looks like the following, where the catalog.json files under the landsat-8-l1 and sentinel-2-l1c are `Collections`, the rest of the catalog.json files are simple `Catalogs`, and the item.json files are `Items`.

**Test Catalog structure:**

```
catalog
├── catalog.json
└── eo
    ├── catalog.json
    ├── landsat-8-l1
    │   ├── catalog.json
    │   └── item.json
    └── sentinel-2-l1c
        ├── catalog.json
        └── sentinel-2a
            ├── catalog.json
            └── item.json
```


## STAC classes

The main STAC classes: Catalog, Collection, and Item model the equivalent entities from the STAC spec. They can be opened directly from JSON files or created from JSON-serializable dictionaries.

The Python classes correspond to the different STAC entities, with some additional classes due to implementation.

- **Thing**: A Thing is not a STAC entity, it is a low-level parent class that is used by Catalog, Collection, and Item and includes the attributes they all have in common (read and save JSON, get links).
- **Catalog**: A catalog is the simplest STAC object, containing an id, a description, the STAC version, and a list of links.
- **Collection**: A Collection is a STAC catalog with some additional fields that describe a group of data, such as the provider, license, along with temporal and spatial extent.
- **Item**: The Item class implements the STAC Item, and has several convenience functions such as retrieving the collection, getting assets by common band name (if using the EO Extension)
- **Items**: The Items class does not correspond to a STAC object. It is a FeatureCollection of `Item`s, possibly from multiple collections. It is used to save and load sets of `Item`s as a FeatureCollection file, along with convenience functions for extracting info across the set.

## STAC entities

Every STAC class has an `open` function to open a file, either local or from a remote URL. Every entity has links which are used to traverse the catalog (open up parent catalogs, child items, etc.), see Tutorial-1 for more info on traversing catalogs.

In [1]:
from satstac import Catalog, Collection, Item

cat = Catalog.open('https://landsat-stac.s3.amazonaws.com/catalog.json')
print(cat, cat.filename)

col = Collection.open('https://landsat-stac.s3.amazonaws.com/landsat-8-l1/catalog.json')
print(col, col.filename)

item = Item.open('https://landsat-stac.s3.amazonaws.com/landsat-8-l1/111/111/2018-11-30/LC81111112018334LGN00.json')
print(item, item.filename)

landsat-stac https://landsat-stac.s3.amazonaws.com/catalog.json
landsat-8-l1 https://landsat-stac.s3.amazonaws.com/landsat-8-l1/catalog.json
LC81111112018334LGN00 https://landsat-stac.s3.amazonaws.com/landsat-8-l1/111/111/2018-11-30/LC81111112018334LGN00.json


### Catalog

The Catalog class has additional attributes that correspond to fields in the STAC spec for Catalogs.

In [2]:
cat = Catalog.open('https://landsat-stac.s3.amazonaws.com/catalog.json')
print('Description:', cat.description)
print('STAC Version:', cat.stac_version)

Description: STAC for Landsat data
STAC Version: 0.6.0


### Collection

The Collection class has properties on top of what Catalog (Collections are also Catalogs) provides.

In [3]:
print('Title:', col.title)
print('Collection Version:', col.version)
print('Keywords: ', col.keywords)
print('License:', col.license)
print('Providers:', col.providers)
print('Extent', col.extent)

Title: Landsat 8 L1
Collection Version: 0.1.0
Keywords:  ['landsat', 'earth observation', 'usgs']
License: PDDL-1.0
Providers: [{'name': 'USGS', 'roles': ['producer'], 'url': 'https://landsat.usgs.gov/'}, {'name': 'Planet Labs', 'roles': ['processor'], 'url': 'https://github.com/landsat-pds/landsat_ingestor'}, {'name': 'AWS', 'roles': ['host'], 'url': 'https://landsatonaws.com/'}, {'name': 'Development Seed', 'roles': ['processor'], 'url': 'https://github.com/sat-utils/sat-api'}]
Extent {'spatial': [-180, -90, 180, 90], 'temporal': ['2013-06-01', None]}


In addition, Collections may have properties, a dictionary of fields that are shared among all Items that are part of the Collection. The property key can be provided directly as an index to the Collection object.

In [4]:
for key in col.properties:
    print('%s: %s' % (key, col[key]))

collection: landsat-8-l1
eo:gsd: 15
eo:platform: landsat-8
eo:instrument: OLI_TIRS
eo:off_nadir: 0
eo:bands: [{'id': 'B1', 'common_name': 'coastal', 'gsd': 30, 'center_wavelength': 0.44, 'full_width_half_max': 0.02}, {'id': 'B2', 'common_name': 'blue', 'gsd': 30, 'center_wavelength': 0.48, 'full_width_half_max': 0.06}, {'id': 'B3', 'common_name': 'green', 'gsd': 30, 'center_wavelength': 0.56, 'full_width_half_max': 0.06}, {'id': 'B4', 'common_name': 'red', 'gsd': 30, 'center_wavelength': 0.65, 'full_width_half_max': 0.04}, {'id': 'B5', 'common_name': 'nir', 'gsd': 30, 'center_wavelength': 0.86, 'full_width_half_max': 0.03}, {'id': 'B6', 'common_name': 'swir16', 'gsd': 30, 'center_wavelength': 1.6, 'full_width_half_max': 0.08}, {'id': 'B7', 'common_name': 'swir22', 'gsd': 30, 'center_wavelength': 2.2, 'full_width_half_max': 0.2}, {'id': 'B8', 'common_name': 'pan', 'gsd': 15, 'center_wavelength': 0.59, 'full_width_half_max': 0.18}, {'id': 'B9', 'common_name': 'cirrus', 'gsd': 30, 'center

### Item

Items have additional attributes representing the STAC fields of the same names.

In [5]:
print(item.bbox)
print(item.geometry)
print(item.assets)

[89.8765, -73.45899, 98.35741, -70.88738]
{'type': 'Polygon', 'coordinates': [[[89.8765, -73.45899], [89.89249, -71.03905], [97.28782, -70.88738], [98.35741, -73.28388], [89.8765, -73.45899]]]}
{'index': {'type': 'text/html', 'title': 'HTML index page', 'href': 'https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/111/111/LC08_L1GT_111111_20181130_20181130_01_RT/index.html'}, 'thumbnail': {'title': 'Thumbnail image', 'type': 'image/jpeg', 'href': 'https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/111/111/LC08_L1GT_111111_20181130_20181130_01_RT/LC08_L1GT_111111_20181130_20181130_01_RT_thumb_large.jpg'}, 'B1': {'type': 'image/x.geotiff', 'eo:bands': [0], 'title': 'Band 1 (coastal)', 'href': 'https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/111/111/LC08_L1GT_111111_20181130_20181130_01_RT/LC08_L1GT_111111_20181130_20181130_01_RT_B1.TIF'}, 'B2': {'type': 'image/x.geotiff', 'eo:bands': [1], 'title': 'Band 2 (blue)', 'href': 'https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/111/1

In addition to these attributes, the sat-stac Item has some derived attributes. `datetime` is a Python datetime object parsed from the Item's `datetime` property, which is a string. `date` is just the date portion of the `datetime`

In [6]:
print(item.datetime)
print(item.date)

2018-11-30 02:10:47.290416+00:00
2018-11-30


#### Properties

The Item class also has properties and can be indexed by key just like the Collection class.

In [7]:
for key in item.properties:
    print('%s: %s' % (key, item.properties[key]))

collection: landsat-8-l1
datetime: 2018-11-30T02:10:47.290416+00:00
eo:sun_azimuth: 57.94044032
eo:sun_elevation: 32.07614408
eo:cloud_cover: 6
eo:row: 111
eo:column: 111
landsat:product_id: LC08_L1GT_111111_20181130_20181130_01_RT
landsat:scene_id: LC81111112018334LGN00
landsat:processing_level: L1GT
landsat:tier: RT


When indexing an Item for a specific property if the property doesn't exist in the item, the item will return that property for the Collection it belongs to, if it exists. Behind the scenes, sat-stac follows the link in the Item to it's Collection and searches it for the property. Providers are free to put any properties of Items into it's Collection if it applies to all Items (if using the STAC `Commons` extension) so this ensures those properties can be directly accessed from the Item.

In [8]:
print(item['eo:platform'])
print(item['eo:gsd'])

landsat-8
15


To get the entire Collection an Item belongs to use the `collection` function.

In [9]:
_col = item.collection()
print(_col, _col.filename)

landsat-8-l1 https://landsat-stac.s3.amazonaws.com/landsat-8-l1/catalog.json


#### Assets

Finally here's the actual data a STAC Item references. As shown above the `assets` attribute returns the entire dictionary of assets. Below are the asset keys in this Item. Individual assets can be accessed directly using this key.

In [10]:
print(item.assets.keys())
print(item.asset('thumbnail'))

dict_keys(['index', 'thumbnail', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'ANG', 'MTL', 'BQA'])
{'title': 'Thumbnail image', 'type': 'image/jpeg', 'href': 'https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/111/111/LC08_L1GT_111111_20181130_20181130_01_RT/LC08_L1GT_111111_20181130_20181130_01_RT_thumb_large.jpg'}


The `asset` function is not just accessing the assets dictionary however. If the name of a common band is provided (see the STAC EO Extension: https://github.com/radiantearth/stac-spec/tree/master/extensions/eo) it will return an appropriate asset, if it exists.

In [11]:
print(item.asset('red'))

{'type': 'image/x.geotiff', 'eo:bands': [3], 'title': 'Band 4 (red)', 'href': 'https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/111/111/LC08_L1GT_111111_20181130_20181130_01_RT/LC08_L1GT_111111_20181130_20181130_01_RT_B4.TIF'}


Behind the scenes sat-stac looks up the provided name in using the `eo:bands` properties. If there is an asset containing a single band of that common name, it returns it. In other words, this is useful if the Item has assets separated by bands.

#### Downloading Assets

In some applications the assets may be accesed remotely in which case the `href` provides the URL. However, some users may wish to download a copy of one or more assets. The `download` function can be used here. If the file already exists locally it will not be downloaded, unless the overwrite keyword is provided.

In [12]:
filename = item.download('MTL')
print(filename)
filename = item.download('MTL', overwrite=True)
print(filename)

LC81111112018334LGN00_MTL.txt
LC81111112018334LGN00_MTL.txt


By default the files are saved in the current directory. When downloading a lot of files this is hardly desired, so sat-stac provides a way to customize the path and filename assets are saved to. By default the filename is <id>-<key>.<extension>, as shown above but custom patterns can be provided. Tutorial-1 discussed Views, which use custom path and filename patterns when creating Catalogs, and this works much the same way.

In [13]:
filename = item.download('MTL', path='mymetadata/${collection}/${date}')
print(filename)

mymetadata/landsat-8-l1/2018-11-30/LC81111112018334LGN00_MTL.txt


In [14]:
filename = item.download('MTL', path='mymetadata/${collection}', filename='${date}_${id}')
print(filename)

mymetadata/landsat-8-l1/2018-11-30_LC81111112018334LGN00_MTL.txt


What names can be used in the templates? Any property in the Item or Collection can be referenced. In addition:

- **id**: The Item id, as shown above
- **date**: Just the date portion of the `datetime` property, as an ISO formatted string
- **year**: Just the year of the `datetime`
- **month**: Just the month of the `datetime`
- **day**: Just the day of the `datetime`

### Items

The Items class is not a STAC entity. It stores a set of Items, all of the Collections those Items belong to, and optionally a dictionary of a search query that produced the Items. It is primarily used for the storing and loading the results of a search, such as what is produced by sat-search (https://github.com/sat-utils/sat-search).

An Items class can be created directly.

```
from satstac import Items

items = Items(item_list, collection_list, search=search_args)
```

where

- item_list: a list of Item objects
- collection_list: a list of Collection objects that the Item objects belong to
- search_args (optional): a dictionary of the search queries used to produce the Item and Collection objects.

Use the `load` and `save` functions to store and load from from files.

In [15]:
from satstac import Items

items = Items.load('test/items.json')
print(len(items))
print(items[0], items[1])

2
LC81920292019008LGN00 LC81920282019008LGN00


#### Class Functions

The Items class has several functions for accessing info from the member Item objects, with a few functions for accessing information about the search, if there is one.

- dates(): get the set of dates for all Items (in this case there are 2 Landsat scenes from the same date)
- properties(<key>): get the set of properties for all Items

There are two additional functions, bbox() and center(). If the Items object contains an intersect search parameter, they return the bounding box and the center lat/lon coordinates, otherwise they return None.

- bbox(): bounding box of the search
- center(): center coordinates of search

In [16]:
print(items.dates())
print(items.properties('eo:platform'))
print(items.properties('eo:cloud_cover'))

[datetime.date(2019, 1, 8)]
['landsat-8']
[99, 100]


In [17]:
print(items.bbox())
print(items.center())

[11.984710693359375, 44.815941348210835, 12.752380371093748, 45.67740123855739]
[45.24667129338411, 12.368545532226562]


#### Printing Items info

There are two functions for summarizing information about the Items. 

- summary(params=[]): Generates column text of properties for each Item. If params are provided it will print those properties. Any Item or Collection property, along with `date` and `id` can be used, which are the defaults.
- calendar(): Generates a Linux terminal style calendar with dates colored showing which platforms are available for each date. Suitable only for printing to a terminal.

In [18]:
print(items.summary())
print(items.summary(['date', 'id', 'eo:cloud_cover']))

Items (2):
date                      id                        
2019-01-08                LC81920292019008LGN00     
2019-01-08                LC81920282019008LGN00     

Items (2):
date                      id                        eo:cloud_cover            
2019-01-08                LC81920292019008LGN00     100                       
2019-01-08                LC81920282019008LGN00     99                        



In [19]:
print(items.calendar())

                              2019                              

      January               February               March        
Mo Tu We Th Fr Sa Su  Mo Tu We Th Fr Sa Su  Mo Tu We Th Fr Sa Su
    1  2  3  4  5  6               1  2  3               1  2  3  
 7  8  9 10 11 12 13   4  5  6  7  8  9 10   4  5  6  7  8  9 10  
14 15 16 17 18 19 20  11 12 13 14 15 16 17  11 12 13 14 15 16 17  
21 22 23 24 25 26 27  18 19 20 21 22 23 24  18 19 20 21 22 23 24  
28 29 30 31           25 26 27 28           25 26 27 28 29 30 31  

landsat-8 (1)
1 total dates


#### Batch downloading of assets

The Items object has a download() function which can be used to download assets of a given key from all Items.

In [20]:
filenames = items.download('MTL')
print(filenames)

['LC81920292019008LGN00_MTL.txt', 'LC81920282019008LGN00_MTL.txt']


In [21]:
filenames = items.download('MTL', path='downloads/${date}')
print(filenames)

['downloads/2019-01-08/LC81920292019008LGN00_MTL.txt', 'downloads/2019-01-08/LC81920282019008LGN00_MTL.txt']
